# Settings

In [1]:
from dotenv import load_dotenv
import os

# .env load 
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "default"

# check API KEY 
if "OPENAI_API_KEY" not in os.environ:
    print("API KEY 정보가 없습니다. 확인 후 환경변수에 등록해주세요.")

# 1. Langchain 기초

## 1) Langchain 없이 

In [2]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "당신은 USER의 말에 공감해주는 상답사입니다."},
        {"role": "user", "content": "우울해서 빵을 샀어"}
    ]
)

response = completion.choices[0].message
response.content

'저도 때때로 우울할 때 먹는 것으로 위로를 받는 법을 알아요. 빵을 사서 조금이라도 기분이 나아졌으면 좋겠어요. 하지만 더 좋은 방법으로 우울함을 극복하는 방법도 찾아보는 것을 권장해요. 함께 하면 더 좋은 방법을 찾을 수 있을 거예요. 함께 화이팅해요!'

## 2) Langchain 사용

In [8]:
# Model의 구조
from langchain_openai import ChatOpenAI 
model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
print(model)

client=<openai.resources.chat.completions.Completions object at 0x00000147FF5671C0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000147FF5782E0> openai_api_key=SecretStr('**********') openai_proxy=''


### (1) 단순 질의응답

In [3]:
from langchain_openai import ChatOpenAI 

model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
response = model.invoke("오늘 너무 우울해서 빵을 샀어")
print(response.content)

빵을 산 것이 우울함을 달래줄 수 있기를 바랍니다. 그 빵을 맛있게 먹으면서 기분이 조금이라도 나아지길 기대해봐도 좋을 것 같아요. 그리고 우울한 마음을 털어놓고 이야기할 수 있는 친구나 가족과 소통하는 것도 도움이 될 수 있을 거예요. 힘내세요! 함께 응원해요.


### (2) 역할 구분 질의응답

In [9]:
from langchain_openai import ChatOpenAI 
from langchain.schema import AIMessage, HumanMessage, SystemMessage 

model = ChatOpenAI(model_name="gpt-3.5-turbo")
messages = [
    SystemMessage(
        content = "당신은 매우 이성적인 사람입니다."
    ),
    HumanMessage(
        content = "오늘 너무 우울해서 빵을 샀어"
    ),
]

response = model.invoke(messages)
response

AIMessage(content='우울할 때 당황스럽고 위압감을 느낄 수 있지만, 먹는 것으로 감정을 달래는 것은 일시적인 해결책이 될 수 있습니다. 좀 더 건강한 방법으로 우울함을 극복할 수 있는 방법을 찾아보는 것이 중요합니다. 운동을 하거나 취미 활동을 즐기는 것도 좋은 방법일 수 있습니다. 또한, 친구나 가족과 소통하거나 전문가의 도움을 받는 것도 고려해보세요. 심리적인 안정을 찾는 것이 중요합니다. 함께 이야기를 나누는 것이 도움이 되길 바랍니다.', response_metadata={'token_usage': {'completion_tokens': 222, 'prompt_tokens': 46, 'total_tokens': 268}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c152843a-fa60-4f5e-a3d2-b239777dbacc-0')

# 2. Langchain + Prompt

In [2]:
from langchain_openai import ChatOpenAI 
from langchain.chains import LLMChain

def make_chain(prompt):
    model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
    chain = LLMChain(prompt=prompt, llm=model)

    return chain

## 1) PromptTemplate

### EXAMPLE 1

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{object}가 영어로 뭐야?")
print(prompt)

input_variables=['object'] template='{object}가 영어로 뭐야?'


In [5]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐"})
response

{'object': '다람쥐', 'text': 'squirrel'}

### EXAMPLE2

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{object}가 {language}로 뭐야?")
print(prompt)

input_variables=['language', 'object'] template='{object}가 {language}로 뭐야?'


In [9]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐", "language":"영어"})
print(response)

{'object': '다람쥐', 'language': '영어', 'text': 'squirrel'}

## 2) ChatPromptTemplate

### EXAMPLE 1

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영어로 번역해주세요"),
        ("human", "{object}"),
    ]
)
print(prompt)

input_variables=['object'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='영어로 번역해주세요')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['object'], template='{object}'))]


In [15]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐"})
print(response)

{'object': '다람쥐', 'text': 'Squirrel'}


### EXAMPLE 2

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "{language}로 번역해주세요"),
        ("human", "{object}"),
    ]
)
print(prompt)

input_variables=['language', 'object'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='{language}로 번역해주세요')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['object'], template='{object}'))]


In [17]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐", "language":"영어"})
response

{'object': '다람쥐', 'language': '영어', 'text': 'squirrel'}

## 3) 반복 작업

In [29]:
input_list = [{"object":"선풍기"},{"object":"지하철"},{"object":"번역"}]

In [33]:
prompt1 = PromptTemplate.from_template("{object} 가 영어로 뭐야?")
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", "영어로 번역해주세요"),
        ("human", "{object}"),
    ]
)

### EXAMPLE 1 - apply

In [34]:
# Prompt 1
chain = make_chain(prompt1)

response = chain.apply(input_list)
print(response)


[{'text': '선풍기는 영어로 "fan" 입니다.'},
 {'text': 'Subway'},
 {'text': '번역은 "translation" 입니다.'}]

In [35]:
# Prompt 2
chain = make_chain(prompt2)

response = chain.apply(input_list)
print(response)

[{'text': 'fan'}, {'text': 'subway'}, {'text': 'Translation'}]

### EXAMPLE 2 - generate

In [45]:
# Prompt 1
chain = make_chain(prompt1)

response = chain.generate(input_list)
print([x[0].text for x in response.generations])

['선풍기는 영어로 "fan" 입니다.', 'Subway', '번역은 영어로 translation이라고 합니다.']


In [46]:
# Prompt 2
chain = make_chain(prompt2)

response = chain.generate(input_list)
print([x[0].text for x in response.generations])

['fan', 'Subway', 'Translation']
